In [1]:
import cv2
import numpy as np
from math import acos, pi, sqrt, cos

In [2]:
# B,G,R
def compute_Hue(B, G, R):
    # prevent divide by 0, adding 0.00000001
    angle = 0.5*((R - G) + (R - B)) / sqrt((R-G)*(R-G) + (R-B)*(G-B) + 0.00000001)
    if B<= G:
        return acos(angle)
    elif B> G:
        return (2*pi- acos(angle))
    #return acos(angle) if B <= G else (2*pi - acos(angle))

In [46]:
# Indensity만 변경

src = cv2.imread('test image/pepper.bmp', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
        
negative = np.zeros((height, width,3), dtype=np.uint8)
plt = np.zeros((height, width,3), dtype=np.uint8)
HSI = np.zeros((height, width,3), dtype=np.uint8)


I = np.zeros((height, width))
S = np.zeros((height, width))
H = np.zeros((height, width))
hsi2B_negative = np.zeros((height, width))
hsi2G_negative = np.zeros((height, width))
hsi2R_negative = np.zeros((height, width))


#B, G, R = cv2.split(src)
B = src[:,:,0]
G = src[:,:,1]
R = src[:,:,2]



for i in range(height):
    for j in range(width):
        B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
        I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
        # prevent divide by 0, adding 0.00000001
        S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
        H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

        
#Negative
for gamma in [0.2, 0.45, 1.5, 4.0]:
    hsi2B_power = np.zeros((height, width))
    hsi2G_power = np.zeros((height, width))
    hsi2R_power = np.zeros((height, width))
    for i in range(height):
        for j in range(width):
            if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                hsi2B_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_negative[i][j] = (I[i][j]*3. - (hsi2R_negative[i][j]+hsi2B_negative[i][j]))
                
                I[i][j] = I[i][j] ** gamma
                hsi2B_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_power[i][j] = (I[i][j]*3. - (hsi2R_power[i][j]+hsi2B_power[i][j]))
                I[i][j] = I[i][j] ** (1/gamma)
            
            if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                H[i][j] = H[i][j]-2*pi/3
                hsi2R_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_negative[i][j] = (I[i][j]*3 - (hsi2R_negative[i][j]+hsi2G_negative[i][j]))
                
                I[i][j] = I[i][j] ** gamma
                hsi2R_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_power[i][j] = (I[i][j]*3 - (hsi2R_power[i][j]+hsi2G_power[i][j]))
                I[i][j] = I[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+2*pi/3
        
            if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                H[i][j] = H[i][j] - 4*pi/3
                I[i][j] = 1 - I[i][j]
                hsi2G_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_negative[i][j] = (I[i][j]*3 - (hsi2G_negative[i][j]+hsi2B_negative[i][j]))
                I[i][j] = 1 - I[i][j]
                
                I[i][j] = I[i][j] ** gamma
                hsi2G_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_power[i][j] = (I[i][j]*3 - (hsi2G_power[i][j]+hsi2B_power[i][j]))
                I[i][j] = I[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+4*pi/3

    plt = cv2.merge((hsi2B_power, hsi2G_power, hsi2R_power))
    cv2.imshow("lambda "+str(gamma), plt)            

negative = cv2.merge((1-hsi2B_negative,1-hsi2G_negative,1-hsi2R_negative))


cv2.imshow("negative", negative)
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
# Saturation 변경

src = cv2.imread('test image/pepper.bmp', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
            
negative = np.zeros((height, width,3), dtype=np.uint8)
plt = np.zeros((height, width,3), dtype=np.uint8)
HSI = np.zeros((height, width,3), dtype=np.uint8)


I = np.zeros((height, width))
S = np.zeros((height, width))
H = np.zeros((height, width))
hsi2B_negative = np.zeros((height, width))
hsi2G_negative = np.zeros((height, width))
hsi2R_negative = np.zeros((height, width))


#B, G, R = cv2.split(src)
B = src[:,:,0]
G = src[:,:,1]
R = src[:,:,2]



for i in range(height):
    for j in range(width):
        B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
        I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
        # prevent divide by 0, adding 0.00000001
        S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
        H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

        
#Negative
for gamma in [0.2, 0.45, 1.5, 4.0]:
    hsi2B_power = np.zeros((height, width))
    hsi2G_power = np.zeros((height, width))
    hsi2R_power = np.zeros((height, width))
    for i in range(height):
        for j in range(width):
            if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                hsi2B_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_negative[i][j] = (I[i][j]*3. - (hsi2R_negative[i][j]+hsi2B_negative[i][j]))
                
                S[i][j] = S[i][j] ** gamma
                hsi2B_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_power[i][j] = (I[i][j]*3. - (hsi2R_power[i][j]+hsi2B_power[i][j]))
                S[i][j] = S[i][j] ** (1/gamma)
            
            if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                H[i][j] = H[i][j]-2*pi/3
                hsi2R_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_negative[i][j] = (I[i][j]*3 - (hsi2R_negative[i][j]+hsi2G_negative[i][j]))
                
                S[i][j] = S[i][j] ** gamma
                hsi2R_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_power[i][j] = (I[i][j]*3 - (hsi2R_power[i][j]+hsi2G_power[i][j]))
                S[i][j] = S[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+2*pi/3
        
            if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                H[i][j] = H[i][j] - 4*pi/3
                hsi2G_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_negative[i][j] = (I[i][j]*3 - (hsi2G_negative[i][j]+hsi2B_negative[i][j]))
                
                S[i][j] = S[i][j] ** gamma
                hsi2G_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_power[i][j] = (I[i][j]*3 - (hsi2G_power[i][j]+hsi2B_power[i][j]))
                S[i][j] = S[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+4*pi/3

    plt = cv2.merge((hsi2B_power, hsi2G_power, hsi2R_power))
    cv2.imshow("lambda "+str(gamma), plt)            

negative = cv2.merge((1-hsi2B_negative,1-hsi2G_negative,1-hsi2R_negative))


cv2.imshow("negative", negative)
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
# Hue 변경

src = cv2.imread('test image/bridge.bmp', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
            
negative = np.zeros((height, width,3), dtype=np.uint8)
plt = np.zeros((height, width,3), dtype=np.uint8)
HSI = np.zeros((height, width,3), dtype=np.uint8)


I = np.zeros((height, width))
S = np.zeros((height, width))
H = np.zeros((height, width))
hsi2B_negative = np.zeros((height, width))
hsi2G_negative = np.zeros((height, width))
hsi2R_negative = np.zeros((height, width))


#B, G, R = cv2.split(src)
B = src[:,:,0]
G = src[:,:,1]
R = src[:,:,2]



for i in range(height):
    for j in range(width):
        B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
        I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
        # prevent divide by 0, adding 0.00000001
        S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
        H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

        
#Negative
for gamma in [0.2, 0.45, 1.5, 4.0]:
    hsi2B_power = np.zeros((height, width))
    hsi2G_power = np.zeros((height, width))
    hsi2R_power = np.zeros((height, width))
    for i in range(height):
        for j in range(width):
            if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                hsi2B_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_negative[i][j] = (I[i][j]*3. - (hsi2R_negative[i][j]+hsi2B_negative[i][j]))
                
                H[i][j] = H[i][j] ** gamma
                hsi2B_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_power[i][j] = (I[i][j]*3. - (hsi2R_power[i][j]+hsi2B_power[i][j]))
                H[i][j] = H[i][j] ** (1/gamma)
            
            if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                H[i][j] = H[i][j]-2*pi/3
                hsi2R_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_negative[i][j] = (I[i][j]*3 - (hsi2R_negative[i][j]+hsi2G_negative[i][j]))
                
                H[i][j] = S[i][j] ** gamma
                hsi2R_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_power[i][j] = (I[i][j]*3 - (hsi2R_power[i][j]+hsi2G_power[i][j]))
                H[i][j] = S[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+2*pi/3
        
            if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                H[i][j] = H[i][j] - 4*pi/3
                hsi2G_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_negative[i][j] = (I[i][j]*3 - (hsi2G_negative[i][j]+hsi2B_negative[i][j]))
                
                H[i][j] = S[i][j] ** gamma
                hsi2G_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_power[i][j] = (I[i][j]*3 - (hsi2G_power[i][j]+hsi2B_power[i][j]))
                H[i][j] = S[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+4*pi/3

    plt = cv2.merge((hsi2B_power, hsi2G_power, hsi2R_power))
    cv2.imshow("lambda "+str(gamma), plt)            

negative = cv2.merge((1-hsi2B_negative,1-hsi2G_negative,1-hsi2R_negative))


cv2.imshow("negative", negative)
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
# Indensity, Saturation 변경

src = cv2.imread('test image/bridge.bmp', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
        
negative = np.zeros((height, width,3), dtype=np.uint8)
plt = np.zeros((height, width,3), dtype=np.uint8)
HSI = np.zeros((height, width,3), dtype=np.uint8)

I = np.zeros((height, width))
S = np.zeros((height, width))
H = np.zeros((height, width))
hsi2B_negative = np.zeros((height, width))
hsi2G_negative = np.zeros((height, width))
hsi2R_negative = np.zeros((height, width))


#B, G, R = cv2.split(src)
B = src[:,:,0]
G = src[:,:,1]
R = src[:,:,2]



for i in range(height):
    for j in range(width):
        B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
        I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
        # prevent divide by 0, adding 0.00000001
        S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
        H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

        
#Negative
for gamma in [0.2, 0.45, 1.5, 4.0]:
    hsi2B_power = np.zeros((height, width))
    hsi2G_power = np.zeros((height, width))
    hsi2R_power = np.zeros((height, width))
    for i in range(height):
        for j in range(width):
            if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                hsi2B_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_negative[i][j] = (I[i][j]*3. - (hsi2R_negative[i][j]+hsi2B_negative[i][j]))
                
                I[i][j] = I[i][j] ** gamma
                S[i][j] = S[i][j] ** gamma
                hsi2B_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_power[i][j] = (I[i][j]*3. - (hsi2R_power[i][j]+hsi2B_power[i][j]))
                I[i][j] = I[i][j] ** (1/gamma)
                S[i][j] = S[i][j] ** (1/gamma)
            
            if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                H[i][j] = H[i][j]-2*pi/3
                hsi2R_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_negative[i][j] = (I[i][j]*3 - (hsi2R_negative[i][j]+hsi2G_negative[i][j]))
                
                I[i][j] = I[i][j] ** gamma
                S[i][j] = S[i][j] ** gamma
                hsi2R_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_power[i][j] = (I[i][j]*3 - (hsi2R_power[i][j]+hsi2G_power[i][j]))
                I[i][j] = I[i][j] ** (1/gamma)
                S[i][j] = S[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+2*pi/3
        
            if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                H[i][j] = H[i][j] - 4*pi/3
                hsi2G_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_negative[i][j] = (I[i][j]*3 - (hsi2G_negative[i][j]+hsi2B_negative[i][j]))
                
                I[i][j] = I[i][j] ** gamma
                S[i][j] = S[i][j] ** gamma
                hsi2G_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_power[i][j] = (I[i][j]*3 - (hsi2G_power[i][j]+hsi2B_power[i][j]))
                I[i][j] = I[i][j] ** (1/gamma)
                S[i][j] = S[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+4*pi/3

    plt = cv2.merge((hsi2B_power, hsi2G_power, hsi2R_power))
    cv2.imshow("lambda "+str(gamma), plt)            

negative = cv2.merge((1-hsi2B_negative,1-hsi2G_negative,1-hsi2R_negative))


cv2.imshow("negative", negative)
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
# Indensity, Hue 변경

src = cv2.imread('test image/pepper.bmp', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
        
negative = np.zeros((height, width,3), dtype=np.uint8)
plt = np.zeros((height, width,3), dtype=np.uint8)
HSI = np.zeros((height, width,3), dtype=np.uint8)


I = np.zeros((height, width))
S = np.zeros((height, width))
H = np.zeros((height, width))
hsi2B_negative = np.zeros((height, width))
hsi2G_negative = np.zeros((height, width))
hsi2R_negative = np.zeros((height, width))


#B, G, R = cv2.split(src)
B = src[:,:,0]
G = src[:,:,1]
R = src[:,:,2]



for i in range(height):
    for j in range(width):
        B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
        I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
        # prevent divide by 0, adding 0.00000001
        S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
        H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

        
#Negative
for gamma in [0.2, 0.45, 1.5, 4.0]:
    hsi2B_power = np.zeros((height, width))
    hsi2G_power = np.zeros((height, width))
    hsi2R_power = np.zeros((height, width))
    for i in range(height):
        for j in range(width):
            if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                hsi2B_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_negative[i][j] = (I[i][j]*3. - (hsi2R_negative[i][j]+hsi2B_negative[i][j]))
                
                I[i][j] = I[i][j] ** gamma
                H[i][j] = H[i][j] ** gamma
                hsi2B_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_power[i][j] = (I[i][j]*3. - (hsi2R_power[i][j]+hsi2B_power[i][j]))
                I[i][j] = I[i][j] ** (1/gamma)
                H[i][j] = H[i][j] ** (1/gamma)
            
            if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                H[i][j] = H[i][j]-2*pi/3
                hsi2R_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_negative[i][j] = (I[i][j]*3 - (hsi2R_negative[i][j]+hsi2G_negative[i][j]))
                
                I[i][j] = I[i][j] ** gamma
                H[i][j] = H[i][j] ** gamma
                hsi2R_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_power[i][j] = (I[i][j]*3 - (hsi2R_power[i][j]+hsi2G_power[i][j]))
                I[i][j] = I[i][j] ** (1/gamma)
                H[i][j] = H[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+2*pi/3
        
            if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                H[i][j] = H[i][j] - 4*pi/3
                hsi2G_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_negative[i][j] = (I[i][j]*3 - (hsi2G_negative[i][j]+hsi2B_negative[i][j]))
                
                I[i][j] = I[i][j] ** gamma
                H[i][j] = H[i][j] ** gamma
                hsi2G_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_power[i][j] = (I[i][j]*3 - (hsi2G_power[i][j]+hsi2B_power[i][j]))
                I[i][j] = I[i][j] ** (1/gamma)
                H[i][j] = H[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+4*pi/3

    plt = cv2.merge((hsi2B_power, hsi2G_power, hsi2R_power))
    cv2.imshow("lambda "+str(gamma), plt)            

negative = cv2.merge((1-hsi2B_negative,1-hsi2G_negative,1-hsi2R_negative))


cv2.imshow("negative", negative)
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
# Saturation, Hue 변경

src = cv2.imread('test image/bridge.bmp', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
        
negative = np.zeros((height, width,3), dtype=np.uint8)
plt = np.zeros((height, width,3), dtype=np.uint8)
HSI = np.zeros((height, width,3), dtype=np.uint8)


I = np.zeros((height, width))
S = np.zeros((height, width))
H = np.zeros((height, width))
hsi2B_negative = np.zeros((height, width))
hsi2G_negative = np.zeros((height, width))
hsi2R_negative = np.zeros((height, width))


#B, G, R = cv2.split(src)
B = src[:,:,0]
G = src[:,:,1]
R = src[:,:,2]



for i in range(height):
    for j in range(width):
        B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
        I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
        # prevent divide by 0, adding 0.00000001
        S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
        H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

        
#Negative
for gamma in [0.2, 0.45, 1.5, 4.0]:
    hsi2B_power = np.zeros((height, width))
    hsi2G_power = np.zeros((height, width))
    hsi2R_power = np.zeros((height, width))
    for i in range(height):
        for j in range(width):
            if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                hsi2B_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_negative[i][j] = (I[i][j]*3. - (hsi2R_negative[i][j]+hsi2B_negative[i][j]))
                
                S[i][j] = S[i][j] ** gamma
                H[i][j] = H[i][j] ** gamma
                hsi2B_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_power[i][j] = (I[i][j]*3. - (hsi2R_power[i][j]+hsi2B_power[i][j]))
                S[i][j] = S[i][j] ** (1/gamma)
                H[i][j] = H[i][j] ** (1/gamma)
            
            if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                H[i][j] = H[i][j]-2*pi/3
                hsi2R_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_negative[i][j] = (I[i][j]*3 - (hsi2R_negative[i][j]+hsi2G_negative[i][j]))
                
                S[i][j] = S[i][j] ** gamma
                H[i][j] = H[i][j] ** gamma
                hsi2R_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_power[i][j] = (I[i][j]*3 - (hsi2R_power[i][j]+hsi2G_power[i][j]))
                S[i][j] = S[i][j] ** (1/gamma)
                H[i][j] = H[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+2*pi/3
        
            if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                H[i][j] = H[i][j] - 4*pi/3
                hsi2G_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_negative[i][j] = (I[i][j]*3 - (hsi2G_negative[i][j]+hsi2B_negative[i][j]))
                
                S[i][j] = S[i][j] ** gamma
                H[i][j] = H[i][j] ** gamma
                hsi2G_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_power[i][j] = (I[i][j]*3 - (hsi2G_power[i][j]+hsi2B_power[i][j]))
                S[i][j] = S[i][j] ** (1/gamma)
                H[i][j] = H[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+4*pi/3

    plt = cv2.merge((hsi2B_power, hsi2G_power, hsi2R_power))
    cv2.imshow("lambda "+str(gamma), plt)            

negative = cv2.merge((1-hsi2B_negative,1-hsi2G_negative,1-hsi2R_negative))


cv2.imshow("negative", negative)
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
# Indensity, Saturation, Hue 변경

src = cv2.imread('test image/bridge.bmp', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
         
negative = np.zeros((height, width,3), dtype=np.uint8)
plt = np.zeros((height, width,3), dtype=np.uint8)
HSI = np.zeros((height, width,3), dtype=np.uint8)


I = np.zeros((height, width))
S = np.zeros((height, width))
H = np.zeros((height, width))
hsi2B_negative = np.zeros((height, width))
hsi2G_negative = np.zeros((height, width))
hsi2R_negative = np.zeros((height, width))


#B, G, R = cv2.split(src)
B = src[:,:,0]
G = src[:,:,1]
R = src[:,:,2]



for i in range(height):
    for j in range(width):
        B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
        I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
        # prevent divide by 0, adding 0.00000001
        S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
        H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

        
#Negative
for gamma in [0.2, 0.45, 1.5, 4.0]:
    hsi2B_power = np.zeros((height, width))
    hsi2G_power = np.zeros((height, width))
    hsi2R_power = np.zeros((height, width))
    for i in range(height):
        for j in range(width):
            if H[i][j] >= 0 and H[i][j] < 2*pi/3: #0도~120도
                hsi2B_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_negative[i][j] = (I[i][j]*3. - (hsi2R_negative[i][j]+hsi2B_negative[i][j]))
                
                I[i][j] = I[i][j] ** gamma
                S[i][j] = S[i][j] ** gamma
                H[i][j] = H[i][j] ** gamma
                hsi2B_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2R_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2G_power[i][j] = (I[i][j]*3. - (hsi2R_power[i][j]+hsi2B_power[i][j]))
                I[i][j] = I[i][j] ** (1/gamma)
                S[i][j] = S[i][j] ** (1/gamma)
                H[i][j] = H[i][j] ** (1/gamma)
            
            if H[i][j] >= 2*pi/3 and H[i][j] < 4*pi/3:
                H[i][j] = H[i][j]-2*pi/3
                hsi2R_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_negative[i][j] = (I[i][j]*3 - (hsi2R_negative[i][j]+hsi2G_negative[i][j]))
                
                I[i][j] = I[i][j] ** gamma
                S[i][j] = S[i][j] ** gamma
                H[i][j] = H[i][j] ** gamma
                hsi2R_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2G_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2B_power[i][j] = (I[i][j]*3 - (hsi2R_power[i][j]+hsi2G_power[i][j]))
                I[i][j] = I[i][j] ** (1/gamma)
                S[i][j] = S[i][j] ** (1/gamma)
                H[i][j] = H[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+2*pi/3
        
            if H[i][j] >= 4*pi/3 and H[i][j] <= 2*pi:
                H[i][j] = H[i][j] - 4*pi/3
                hsi2G_negative[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_negative[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_negative[i][j] = (I[i][j]*3 - (hsi2G_negative[i][j]+hsi2B_negative[i][j]))
                
                I[i][j] = I[i][j] ** gamma
                S[i][j] = S[i][j] ** gamma
                H[i][j] = H[i][j] ** gamma
                hsi2G_power[i][j] = (1-S[i][j])*I[i][j]
                hsi2B_power[i][j] = (1+(S[i][j]*cos(H[i][j]) / cos(pi/3-H[i][j])))*I[i][j]
                hsi2R_power[i][j] = (I[i][j]*3 - (hsi2G_power[i][j]+hsi2B_power[i][j]))
                I[i][j] = I[i][j] ** (1/gamma)
                S[i][j] = S[i][j] ** (1/gamma)
                H[i][j] = H[i][j] ** (1/gamma)
                
                H[i][j] = H[i][j]+4*pi/3

    plt = cv2.merge((hsi2B_power, hsi2G_power, hsi2R_power))
    cv2.imshow("lambda "+str(gamma), plt)            

negative = cv2.merge((1-hsi2B_negative,1-hsi2G_negative,1-hsi2R_negative))


cv2.imshow("negative", negative)
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
# histogram - low
def histogram_low(value):
    height, width = value.shape[0], value.shape[1]
    value_low = 255
    value_high = 0
    for i in range(height):
        for j in range(width):
            if value_low >= value[i][j]:
                value_low = value[i][j]
            if value_high <= value[i][j]:
                value_high = value[i][j]
    
    for i in range(height):
        for j in range(width):
            if value[i][j] <= value_low:
                value[i][j] = 0
            elif value[i][j] > value_low and value[i][j] < value_high:
                value[i][j] = 255*(value[i][j] - value_low)/(value_high - value_low)
            else: #value[i][j] >= value_high
                value[i][j] = 255
                
    return value

In [45]:
# Implementation of Histogram Equalization

src = cv2.imread('test image/dark.jpg', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
            
hist = np.zeros((height, width,3), dtype=np.uint8)
inv_hist = np.zeros((height, width,3), dtype=np.uint8)

I = np.zeros((height, width))
S = np.zeros((height, width))
H = np.zeros((height, width))
inv_I = np.zeros((height, width))
inv_S = np.zeros((height, width))
inv_H = np.zeros((height, width))
hsi2B = np.zeros((height, width))
hsi2G = np.zeros((height, width))
hsi2R = np.zeros((height, width))

B = src[:,:,0]
G = src[:,:,1]
R = src[:,:,2]

for i in range(height):
    for j in range(width):
        B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
        I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
        S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
        H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

I = I*255

inv_H = H
inv_S = S
inv_I = histogram_low(I)/255

for i in range(height):
    for j in range(width):
        if inv_H[i][j] >= 0 and inv_H[i][j] < 2*pi/3: #0도~120도
            hsi2B[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2R[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2G[i][j] = (inv_I[i][j]*3 - (hsi2R[i][j]+hsi2B[i][j]))
                
        if inv_H[i][j] >= 2*pi/3 and inv_H[i][j] < 4*pi/3:
            inv_H[i][j] = inv_H[i][j]-2*pi/3
            hsi2R[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2G[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2B[i][j] = (inv_I[i][j]*3 - (hsi2R[i][j]+hsi2G[i][j]))
                
        if inv_H[i][j] >= 4*pi/3 and inv_H[i][j] <= 2*pi:
            inv_H[i][j] = inv_H[i][j] - 4*pi/3
            hsi2G[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2B[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2R[i][j] = (inv_I[i][j]*3 - (hsi2G[i][j]+hsi2B[i][j]))
                    
inv_hist = cv2.merge((hsi2B, hsi2G, hsi2R))         

cv2.imshow("1st histogram_", inv_hist)
cv2.imshow("1st photo", src)

In [39]:
# Implementation of Histogram Equalization

src = cv2.imread('HE test/Fig0316(1)(top_left).jpg', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
            
hist = np.zeros((height, width,3), dtype=np.uint8)
inv_hist = np.zeros((height, width,3), dtype=np.uint8)

I = np.zeros((height, width))
S = np.zeros((height, width))
H = np.zeros((height, width))
inv_I = np.zeros((height, width))
inv_S = np.zeros((height, width))
inv_H = np.zeros((height, width))
hsi2B = np.zeros((height, width))
hsi2G = np.zeros((height, width))
hsi2R = np.zeros((height, width))

B = src[:,:,0]
G = src[:,:,1]
R = src[:,:,2]

for i in range(height):
    for j in range(width):
        B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
        I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
        S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
        H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

I = I*255

inv_H = H
inv_S = S
inv_I = histogram_low(I)/255

for i in range(height):
    for j in range(width):
        if inv_H[i][j] >= 0 and inv_H[i][j] < 2*pi/3: #0도~120도
            hsi2B[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2R[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2G[i][j] = (inv_I[i][j]*3 - (hsi2R[i][j]+hsi2B[i][j]))
                
        if inv_H[i][j] >= 2*pi/3 and inv_H[i][j] < 4*pi/3:
            inv_H[i][j] = inv_H[i][j]-2*pi/3
            hsi2R[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2G[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2B[i][j] = (inv_I[i][j]*3 - (hsi2R[i][j]+hsi2G[i][j]))
                
        if inv_H[i][j] >= 4*pi/3 and inv_H[i][j] <= 2*pi:
            inv_H[i][j] = inv_H[i][j] - 4*pi/3
            hsi2G[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2B[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2R[i][j] = (inv_I[i][j]*3 - (hsi2G[i][j]+hsi2B[i][j]))
                    
inv_hist = cv2.merge((hsi2B, hsi2G, hsi2R))         

cv2.imshow("1st histogram_", inv_hist)
cv2.imshow("1st photo", src)

In [40]:
# Implementation of Histogram Equalization

src = cv2.imread('HE test/Fig0316(2)(2nd_from_top).jpg', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
            
hist = np.zeros((height, width,3), dtype=np.uint8)
inv_hist = np.zeros((height, width,3), dtype=np.uint8)

I = np.zeros((height, width))
S = np.zeros((height, width))
H = np.zeros((height, width))
inv_I = np.zeros((height, width))
inv_S = np.zeros((height, width))
inv_H = np.zeros((height, width))
hsi2B = np.zeros((height, width))
hsi2G = np.zeros((height, width))
hsi2R = np.zeros((height, width))

B = src[:,:,0]
G = src[:,:,1]
R = src[:,:,2]

for i in range(height):
    for j in range(width):
        B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
        I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
        S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
        H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

I = I*255

inv_H = H
inv_S = S
inv_I = histogram_low(I)/255

for i in range(height):
    for j in range(width):
        if inv_H[i][j] >= 0 and inv_H[i][j] < 2*pi/3: #0도~120도
            hsi2B[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2R[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2G[i][j] = (inv_I[i][j]*3 - (hsi2R[i][j]+hsi2B[i][j]))
                
        if inv_H[i][j] >= 2*pi/3 and inv_H[i][j] < 4*pi/3:
            inv_H[i][j] = inv_H[i][j]-2*pi/3
            hsi2R[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2G[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2B[i][j] = (inv_I[i][j]*3 - (hsi2R[i][j]+hsi2G[i][j]))
                
        if inv_H[i][j] >= 4*pi/3 and inv_H[i][j] <= 2*pi:
            inv_H[i][j] = inv_H[i][j] - 4*pi/3
            hsi2G[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2B[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2R[i][j] = (inv_I[i][j]*3 - (hsi2G[i][j]+hsi2B[i][j]))
                    
inv_hist = cv2.merge((hsi2B, hsi2G, hsi2R))         

cv2.imshow("2nd histogram", inv_hist)
cv2.imshow("2nd photo", src)

In [41]:
# Implementation of Histogram Equalization

src = cv2.imread('HE test/Fig0316(3)(third_from_top).jpg', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
            
hist = np.zeros((height, width,3), dtype=np.uint8)
inv_hist = np.zeros((height, width,3), dtype=np.uint8)

I = np.zeros((height, width))
S = np.zeros((height, width))
H = np.zeros((height, width))
inv_I = np.zeros((height, width))
inv_S = np.zeros((height, width))
inv_H = np.zeros((height, width))
hsi2B = np.zeros((height, width))
hsi2G = np.zeros((height, width))
hsi2R = np.zeros((height, width))

B = src[:,:,0]
G = src[:,:,1]
R = src[:,:,2]

for i in range(height):
    for j in range(width):
        B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
        I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
        S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
        H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

I = I*255

inv_H = H
inv_S = S
inv_I = histogram_low(I)/255

for i in range(height):
    for j in range(width):
        if inv_H[i][j] >= 0 and inv_H[i][j] < 2*pi/3: #0도~120도
            hsi2B[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2R[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2G[i][j] = (inv_I[i][j]*3 - (hsi2R[i][j]+hsi2B[i][j]))
                
        if inv_H[i][j] >= 2*pi/3 and inv_H[i][j] < 4*pi/3:
            inv_H[i][j] = inv_H[i][j]-2*pi/3
            hsi2R[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2G[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2B[i][j] = (inv_I[i][j]*3 - (hsi2R[i][j]+hsi2G[i][j]))
                
        if inv_H[i][j] >= 4*pi/3 and inv_H[i][j] <= 2*pi:
            inv_H[i][j] = inv_H[i][j] - 4*pi/3
            hsi2G[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2B[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2R[i][j] = (inv_I[i][j]*3 - (hsi2G[i][j]+hsi2B[i][j]))
                    
inv_hist = cv2.merge((hsi2B, hsi2G, hsi2R))         

cv2.imshow("3rd histogram", inv_hist)
cv2.imshow("3rd photo", src)

In [44]:
# Implementation of Histogram Equalization

src = cv2.imread('HE test/Fig0316(4)(bottom_left).jpg', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
            
hist = np.zeros((height, width,3), dtype=np.uint8)
inv_hist = np.zeros((height, width,3), dtype=np.uint8)

I = np.zeros((height, width))
S = np.zeros((height, width))
H = np.zeros((height, width))
inv_I = np.zeros((height, width))
inv_S = np.zeros((height, width))
inv_H = np.zeros((height, width))
hsi2B = np.zeros((height, width))
hsi2G = np.zeros((height, width))
hsi2R = np.zeros((height, width))

B = src[:,:,0]
G = src[:,:,1]
R = src[:,:,2]

for i in range(height):
    for j in range(width):
        B_tmp, G_tmp, R_tmp = B[i][j]/255., G[i][j]/255., R[i][j]/255.
        
        I[i][j] = (B_tmp+G_tmp+R_tmp)/3.
        S[i][j] = 1 - 3*np.min([B_tmp,G_tmp,R_tmp])/(B_tmp+G_tmp+R_tmp+0.00000001)
        H[i][j] = compute_Hue(B_tmp, G_tmp, R_tmp)

I = I*255

inv_H = H
inv_S = S
inv_I = histogram_low(I)/255

for i in range(height):
    for j in range(width):
        if inv_H[i][j] >= 0 and inv_H[i][j] < 2*pi/3: #0도~120도
            hsi2B[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2R[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2G[i][j] = (inv_I[i][j]*3 - (hsi2R[i][j]+hsi2B[i][j]))
                
        if inv_H[i][j] >= 2*pi/3 and inv_H[i][j] < 4*pi/3:
            inv_H[i][j] = inv_H[i][j]-2*pi/3
            hsi2R[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2G[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2B[i][j] = (inv_I[i][j]*3 - (hsi2R[i][j]+hsi2G[i][j]))
                
        if inv_H[i][j] >= 4*pi/3 and inv_H[i][j] <= 2*pi:
            inv_H[i][j] = inv_H[i][j] - 4*pi/3
            hsi2G[i][j] = (1-inv_S[i][j])*inv_I[i][j]
            hsi2B[i][j] = (1+(inv_S[i][j]*cos(inv_H[i][j]) / cos(pi/3-inv_H[i][j])))*inv_I[i][j]
            hsi2R[i][j] = (inv_I[i][j]*3 - (hsi2G[i][j]+hsi2B[i][j]))
                    
inv_hist = cv2.merge((hsi2B, hsi2G, hsi2R))         

cv2.imshow("4th histogram_", inv_hist)
cv2.imshow("4th photo", src)

cv2.waitKey(0)
cv2.destroyAllWindows()